# Introduction to the Solution Building Library

This example notebook demonstrates the basics of using the Intrinsic Solution Building Library.

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [flowstate.intrinsic.ai](https://flowstate.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

## Connecting to a solution

The first step when using the Solution Building Library is always to establish a connection to a deployed/running solution. In a notebook in VS Code, you can achieve this with the method `deployments.connect_to_selected_solution()` which requires that your running solution is selected as the target solution in the Intrinsic extension for VS Code.

Navigate to the solution selector of the Intrinsic extension for VS Code (by clicking on the **Intrinsic** icon in the activity bar to the far left side of the VS Code window) and make sure your running solution is selected as the target solution. If you need more help with this step, please take a look at the guide for [connecting VS Code to an Intrinsic project](https://flowstate.intrinsic.ai/docs/guides/build_with_code/connect_vscode_to_cloud/).

![Select solution](images/select_solution.png)

![Selected solution indicator](images/selected_solution_indicator.png)

Once your solution is running and selected correctly, you can run the following:

In [ ]:
from intrinsic.solutions import deployments

solution = deployments.connect_to_selected_solution()

You should see:

```
Connecting to deployed solution...
Connected successfully to "<solution_name>(<build>)" at "<host>".
```

Here you can check again that you are connected to the correct solution. If all looks correct, you can move on to exploring the solution.

## Exploring the solution

The `solution` object represents the running solution and is your top-level entry point. Its properties include:

- `solution.executive`: controls the execution of your processes
- `solution.resources`: represents the resources available in the solution
- `solution.skills`: represents the skills available in the solution
- `solution.world`: represents the belief world
- `solution.simulator`: represents the simulation (only available if running in simulation)

We typically define some convenience shortcuts for accessing these. This does not only save us some typing effort but also enables better auto-completion support in VS Code Jupyter:

In [ ]:
executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world
simulator = solution.simulator

Let's take a look at the available skills and resources in the solution. These are populated dynamically when connecting to a solution and are specific to the solution.

In [ ]:
list(skills.get_skill_ids())

In [ ]:
dir(resources)

In VS Code you can also use auto-completion - to a limited extent. First execute the following cell:

In [ ]:
intrinsic_skills = skills.ai.intrinsic

And now try typing `intrinsic_skills.` or `resources.` (followed by <kbd>Ctrl</kbd> + <kbd>Space</kbd> if necessary):

In [ ]:
# Add '.' to the following lines
intrinsic_skills
resources

This form of auto-completion based on runtime state is unreliable and does not work in all contexts. E.g., it might not work in nested contexts (try typing `print(skills.`)  or when accessing nested properties (try typing `skills.ai.intrinsic.`). See the section on language support below on how to get better auto-completion and IDE support.

## Updating the solution object

If you are running interactively in a notebook, some changes to your solution will require you to re-initialize the Solution Building Library for the changes to have an effect in the running notebook. Generally, you can always re-connect to the running solution and thereby update the solution object:

In [ ]:
solution = deployments.connect_to_selected_solution()

# Update any shortcuts to properties and nested properties of the solution
skills = solution.skills
move_robot = skills.ai.intrinsic.move_robot

If you installed a new or modified skill, you can selectively update the skills provider:

In [ ]:
skills.update()

# Update any shortcuts to properties and nested properties of skills
move_robot = skills.ai.intrinsic.move_robot

If you added or removed resources, you can selectively update the resources provider:

In [ ]:
resources.update()

## Language support in VS Code and other IDEs

The Solution Building Library dynamically creates several helper classes when you connect to a solution. For example, `skills.ai.intrinsic.move_robot` is a helper class for the `move_robot` skill which only exists if the skill is installed in the solution. If the skill is present, the class `skills.ai.intrinsic.move_robot` matches the skill version which is currently installed in the solution. I.e., if you connect to different solution, `skills.ai.intrinsic.move_robot` might be different.

Dynamically created classes can be a challenge to work with if you don't know how. In the following we first show a few simple and reliable solutions and then we show how you can get full language support in your IDE by creating stubs for the Solution Building Library.

### Basic language support

To get information about a dynamically created class **you can always** inspect the class object at runtime, e.g., by using `help()` or functions from the `inspect` module:

In [ ]:
help(skills.ai.intrinsic.move_robot)

Some IDEs have functionality to access this information in a more direct way - without manually calling `help()`. In VS Code, you can use the **Jupyter PowerToys** extension:

1. Click on the **Jupyter** icon <img src="https://raw.githubusercontent.com/microsoft/vscode-codicons/6ceb091d5c40da3e5836e3d80b08d3f74efc4cbf/src/icons/notebook.svg" width="25"> in the activity bar (to the far left side of the VS Code window).
1. Place your cursor, e.g., on `move_robot` in the cell below or select any other code in a code cell.
1. See the `help` output shown live under **CONTEXTUAL HELP** in the sidebar (to the left side of the VS Code window).

In [ ]:
# Place your cursor on 'move_robot' and check the contextual help output in the sidebar.
skills.ai.intrinsic.move_robot


You can also check out the [live animation of the contextual help feature](https://github.com/microsoft/vscode-jupyter-powertoys?tab=readme-ov-file#contextual-help) on the Jupyter PowerToys website.

### Advanced language support through Python stubs

To work with the Solution Building Library in a more comfortable way we highly recommend that you create custom [Python stubs](https://typing.readthedocs.io/en/latest/source/stubs.html) for your solution and configure your IDE or type checker to use them. In the following steps we use the example of VS Code but the process is similar for other IDEs and type checkers.

<div class="alert alert-info">

**Important**

The stubs generated by the Solution Building Library are specific to a solution. They match the skills installed in the solution at their respective version. The stubs need to be updated everytime you connect to a different solution and everytime you install a new or modified skill in the solution.

</div>

**Step 1 - Find/configure stub location**

If you use the Intrinsic devcontainer, the stub-related settings should all be set to their defaults and the stub files are expected to be in `${workspaceFolder}/typings`. We recommend that you leave it this way. If you want to check or customize the stub path then you need to check/change the VS Code setting `python.analysis.stubPath`. By default it is set to the value `typings` which is a path relative to your workspace folder.

**Step 2 - Create stubs**

Create stubs for the solution you are currently connected to. Make sure that your are writing the stubs to the location identified/configured in the previous step.

In [ ]:
# Change 'output_path' as needed!
solution.generate_stubs(output_path="typings")

VS Code by default executes Python notebooks in the folder of the respective notebook and Python scripts are always executed in the workspace folder. Thus, when using the VS Code default settings, you typically would use the following output paths:

| Path of notebook or script  | output_path |
| -------- | ------- |
| `${workspaceFolder}/my_notebook.ipynb`  | `typings` |
| `${workspaceFolder}/foo/my_notebook.ipynb`  | `../typings` |
| `${workspaceFolder}/foo/bar/my_notebook.ipynb`  | `../../typings` |
| `${workspaceFolder}/my_script.py`  | `typings` |
| `${workspaceFolder}/foo/my_script.py`  | `typings` |
| `${workspaceFolder}/foo/bar/my_script.py`  | `typings` |

If you are generating stubs from a notebook and something is not working, make sure to check the VS Code setting `jupyter.notebookFileRoot` which controls in which folder notebooks are executed.

**Step 3 - Reload stubs (if required)**

Make sure that your IDE or type checker picks up the changes. VS Code's language server for Python (Pylance) often does not automatically pick up changes to the stub folder. To force a restart of the language server, you can do **any** of the following:

- Reload the VS Code window, e.g., by running the **Developer: Reload Window** command
- Restart the Python language server by running the **Python: Restart Language Server** command (**this command is only visible when a .py-file is opened in the editor**!)
- Restart VS Code

**Step 4 - Verify that stubs are working**

There are various quick ways to confirm that the stubs are being found and used. In VS Code, e.g., in the following cell right-click on `move_robot` and choose **Go to declaration**. If the file `providers.pyi` inside of the stubs folder is opened, everything is setup correctly. Also, you should get a pretty tooltip with parameter hints if you place your cursor inside of the parentheses of `move_robot()` and press <kbd>CTRL</kbd> + <kbd>SPACE</kbd>.

In [ ]:
a = skills.ai.intrinsic.move_robot()

## Executive

The `executive` is the main entrypoint for running skills and processes in the solution. To demonstrate its usage we first create a few sample skills so that we have *something* to execute. The following example skills here move the robot, so it will be easy to see the effect of running them in the Flowstate solution editor. Creating and parameterizing skills is explained in detail in the "skills" example notebook.

In [ ]:
move_robot = skills.ai.intrinsic.move_robot

# Moves the robot to the 'home' pose.
move_skill_1 = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.JOINT,
        )
    ],
    arm_part=world.robot,
)

# Moves the robot to 'view_pose_left'.
move_skill_2 = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_left,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.JOINT,
        )
    ],
    arm_part=world.robot,
)

### Synchronous execution

You can run a single skill like this:

In [ ]:
executive.run(move_skill_2)

 While the cell is executing, you can watch the robot move in the Flowstate solution editor.

 You can also run a sequence of skills:

In [ ]:
executive.run([move_skill_1, move_skill_2])

After the execution has started you will be able to see the sequence of skills that are being executed in the process editor of the Flowstate solution editor. Because we have passed skill instances directly to `executive.run()`, observe that the skills are unnamed and that the process is called `(untitled)`. This is useful for testing, but usually you should wrap skill instances inside of appropriate behavior tree nodes wrapped by a `BehaviorTree` instance at the top-level:

In [ ]:
from intrinsic.solutions import behavior_tree as bt

tree = bt.BehaviorTree(
    name="My first behavior tree",
    root=bt.Sequence(
        [
            bt.Task(action=move_skill_1, name="Some move"),
            bt.Task(action=move_skill_2, name="Another move"),
        ]
    ),
)

executive.run(tree)

This gives you the same capabilities as the process editor of the Flowstate solution editor. E.g., it allows for naming nodes and opens up the possibility to use flow control nodes such as `Branch` or `Loop`. You can find more complex behavior trees in the other example notebooks.

### Asynchronous execution

Executing a behavior tree can take a while and `executive.run()` will block until the execution has finished. If you want to do something during execution, you can use `run_async`. E.g., you can observe the different state transitions inside the executive:

In [ ]:
from intrinsic.executive.proto import behavior_tree_pb2


def print_executive_state():
    print(
        "Executive state:",
        behavior_tree_pb2.BehaviorTree.State.Name(
            executive.operation.metadata.behavior_tree_state
        ),
    )


def print_is_succeeded():
    print(
        "Is succeeded:",
        executive.operation.metadata.behavior_tree_state
        == behavior_tree_pb2.BehaviorTree.SUCCEEDED,
    )


print_executive_state()

executive.run_async(tree)
print_executive_state()

executive.block_until_completed()
print_executive_state()
print_is_succeeded()

Note that `executive.operation.metadata` is a "Protocol Buffer" (proto) message. You can find all about using protos in Python in the official [Python Generated Code Guide]( https://protobuf.dev/reference/python/python-generated/).

Here you can see the first transition to `RUNNING` and, after calling `executive.block_until_completed()`, you can see the transition to `SUCCEEDED`.

You can also interrupt the execution of the behavior tree and resume it. This is done by using `executive.suspend()` and `executive.resume()` as follows:

In [ ]:
executive.run_async(tree)
print_executive_state()

executive.suspend()
print_executive_state()

executive.resume()
print_executive_state()

executive.block_until_completed()
print_executive_state()

`executive.suspend()` waits for the first skill in the behavior tree to finish and then stops the executive. When `executive.resume()` is called the second skill get executed.
Calling `executive.suspend()` while an action is running leads to the executive being in state `SUSPENDING` until the execution of the skill has finished.
Only afterwards does the executive transition to `SUSPENDED` and therefore succeeds the `executive.suspend()` operation and continues with the program.

If your executive ends up in `FAILED` state the errors are displayed automatically inside the notebook.

You can cancel execution immediately (without the option to resume) by using `executive.cancel()` or `executive.cancel_async()`.

Calling `executive.cancel()` while an action is running leads to the executive being in state `CANCELING` until the running skill finishes cancelling (or, if it does not support cancellation, finishes execution as usual). Afterwards, the executive ends in either the state `CANCELED` (if the cancellation was processed) or in `SUCCEEDED`/`FAILED` (if it finished in success/failure before processing the cancellation).

In [ ]:
executive.run_async(tree)
print_executive_state()

executive.cancel()
print_executive_state()

## Resetting

Various components of the solution can be reset separately from each other.

If you have unsaved world modifications as a result of running certain skills or because you edited the belief world you can restore the belief world to its last saved state like this:

In [ ]:
world.reset()

For more ways to interact with the belief world see the `003_world.ipynb` example.

You can reset the simulation manually which is the same as clicking **Reset** in the **Simulator** tab of the [workcell designer](https://developers.intrinsic.ai/guides/workcell_design/workcell_overview) of the Flowstate solution editor. The simulation state will be reset to the state of the **Belief** world.

In [ ]:
simulator.reset()


If you want to restore the initial state of the executive you can reset it. This restores the initial plan and the executive ends up in state `ACCEPTED` after this.

In [ ]:
executive.reset()
print_executive_state()

## Next steps

Take a look at the following example notebooks to learn:

- How to [interact with the world](003_world.ipynb).
- How to [parameterize skill instances](004_skills.ipynb).
- How to create behavior trees with control flow nodes such as [sequences](005_sequence.ipynb), [loops and branches](006_loop_and_branch.ipynb) or [retries](007_retry.ipynb).